In [1]:
from src.data_loader import csv_to_npy, load_trace, load_trace_CW305, AES_VERSIONS, IS_RAND, RAW_DATASET_PATH, DATASET_PATH, DATASET_PATH_CW305
from sklearn.utils import shuffle

In [2]:
n_train   = 8000
n_val     = 1000
n_test    = 1000
n_samples = 10000

In [3]:
import numpy as np
import os

def save_supervised_data(normal, trojan, version, output_dir="./supervised_dataset", seed=2025):
    print(f"\n--- {version} ---")
    
    # 1. Train Set (Disabled 8000 + Triggered 8000)
    X_train_unshuffled = np.concatenate([normal[:n_train], trojan[:n_train]])
    y_train_unshuffled = np.concatenate([np.zeros(n_train), np.ones(n_train)])
    X_train, y_train = shuffle(X_train_unshuffled, y_train_unshuffled, random_state=seed)

    # 2. Validation Set (Disabled 1000 + Triggered 1000)
    X_val = np.concatenate([normal[n_train:n_train+n_val], trojan[n_train:n_train+n_val]])
    y_val = np.concatenate([np.zeros(n_val), np.ones(n_val)])
    
    # 3. Test Set (Disabled 1000 + Triggered 1000)
    X_test = np.concatenate([normal[n_train+n_val:], trojan[n_train+n_val:]])
    y_test = np.concatenate([np.zeros(n_test), np.ones(n_test)])

    # 4. Save Data & Label
    np.save(os.path.join(output_dir, f'{version}_train.npy'), X_train)
    np.save(os.path.join(output_dir, f'{version}_train_labels.npy'), y_train)
    
    np.save(os.path.join(output_dir, f'{version}_val.npy'), X_val)
    np.save(os.path.join(output_dir, f'{version}_val_labels.npy'), y_val)

    np.save(os.path.join(output_dir, f'{version}_test.npy'), X_test)
    np.save(os.path.join(output_dir, f'{version}_test_labels.npy'), y_test)
    
    print(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")
    print("Save Finished.")

In [ ]:
for version in AES_VERSIONS:
    OUTPUT_DIR = "./supervised_dataset"
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    
    trace = load_trace(version)
    normal_data = trace[:n_samples]
    trojan_data = trace[n_samples:]
    
    save_supervised_data(normal_data, trojan_data, version=version, output_dir=OUTPUT_DIR)